In [747]:
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

with open('jsonFiles/monsters.json') as f:
    data = json.load(f)

#pprint(data) # Prints out all of the Json File

numMonst = 325;
monstNames = np.chararray((numMonst,1))
monstNames = [None]*numMonst
monstCRs = np.zeros((numMonst,1))

# Monsters have up to 7 actions
# Monsters have up to 3 special abilities

for i in range(0,numMonst):
    monstNames[i] = data[i]["name"]
    monstCRs[i] = eval( data[i]["challenge_rating"] )
    #print len(data[i]["special_abilities"])
    #print(data[i]["challenge_rating"])


def plot_hist_of_CRs():
    gaussian_numbers = np.random.randn(1000)
    plt.hist(monstCRs,10)
    plt.title("CR Histogram")
    plt.xlabel("CR")
    plt.ylabel("Frequency")
    plt.show()
    
def print_random_monst_data():
    randomMonstIndex = np.random.randint(numMonst);
    pprint(data[randomMonstIndex])
    i = randomMonstIndex
    
def print_specific_monst_data( index=0 ):
    i = index
    print ('******** '+data[i]['name']+' ******** ')
    print ('________________________________________')
    pprint(data[i])
    
class Monster:
    kind = 'monster'         # class variable shared by all instances

    def __init__(self, is_random=False):
        
        if is_random:
            randomMonstIndex = np.random.randint(numMonst);
            i = randomMonstIndex
        else:
            i = 0
        
        self.canSwim = False
        self.canFly = False
        
        self.name = data[i]['name']    # instance variable unique to each instance
        self.typ = data[i]['type']
        self.subtype = data[i]['subtype']
        self.cr = data[i]['challenge_rating']
        self.alignment = data[i]['alignment']
        self.size = data[i]['size']
        self.index = monstNames.index(self.name)
        
        self.intelligence = data[i]['intelligence']
        self.wisdom = data[i]['wisdom']
        self.strength = data[i]['strength']
        self.dexterity = data[i]['dexterity']
        self.charisma = data[i]['charisma']
        self.constitution = data[i]['constitution']
        
        self.hit_dice = data[i]['hit_dice']
        self.hit_points = data[i]['hit_points']
        self.armor_class = data[i]['armor_class']

        self.speed = data[i]['speed']
        self.senses = data[i]['senses']
        self.languages = data[i]['languages']

        self.condition_immunities = data[i]['condition_immunities']
        self.damage_immunities = data[i]['damage_immunities']
        self.damage_resistances = data[i]['damage_resistances']
        self.damage_vulnerabilities = data[i]['damage_vulnerabilities']
        
        try: # Not all monsters have stealth
            self.stealth = data[i]['stealth']
        except Exception, e:
            #print repr(e)
            self.stealth = 0
            
        # ACTIONS
        self.num_actions = 0
        self.actions_attack_bonus = []
        self.actions_damage_bonus = []
        self.actions_desc = []
        self.actions_name = []
        self.actions_damage_dice = []
        try: # Not all monsters have actions
            self.num_actions = len(data[i]['actions'])
            self.actions = data[i]['actions']
            for j in range(0,self.num_actions):
                self.actions_attack_bonus.append(self.actions[j]['attack_bonus'])
                self.actions_desc.append(self.actions[j]['desc'])
                self.actions_name.append(self.actions[j]['name'])

                # Some special abilities do not have damage dice
                try:
                    self.actions_damage_dice.append(self.actions[j]['damage_dice'])
                except Exception, e:
                    self.actions_damage_dice.append('0d0')
                # Some special abilities do not have damage bonus
                try:
                    self.actions_damage_bonus.append(self.actions[j]['damage_bonus'])
                except Exception, e:
                    self.actions_damage_bonus.append(0)
                    
        except Exception, e:
            #print repr(e)
            #print self.name+' has NO ACTIONS'
            self.actions = None
        
        # SPECIAL ABILITIES
        self.num_special_abilities = 0
        self.special_abilities_name = []
        self.special_abilities_desc = []
        self.special_abilities_attack_bonus = []
        try: # Not all monsters have special abilities
            self.num_special_abilities = len(data[i]['special_abilities'])
            self.special_abilities = data[i]['special_abilities']
            for j in range(0,self.num_special_abilities):
                self.special_abilities_name.append(self.special_abilities[j]['name'])
                self.special_abilities_desc.append(self.special_abilities[j]['desc'])
                self.special_abilities_attack_bonus.append(self.special_abilities[j]['attack_bonus'])
        except Exception, e:
            #print repr(e)
            self.special_abilities = None
            
        if 'fly' in self.speed:
            self.canFly = True
        if 'swim' in self.speed:
            self.canSwim = True
            
    def perform_random_action(self):
        i = np.random.randint( self.num_actions );
        
        name = str( self.name )
        actionName = str( self.actions_name[i] )
        actionDesc = str( self.actions_desc[i] )
        attackBonus = int( self.actions_attack_bonus[i] )
        damageBonus = int( self.actions_damage_bonus[i] )
        
        print('The '+name+' uses its '+actionName)
        print( actionDesc )
        print('')
        
    def perform_random_special_ability(self):
        if self.num_special_abilities == 0:
            print 'No special abilities!'
            print ''
        else:
            i = np.random.randint( self.num_special_abilities );

            name = str( self.name )
            specialAbilityName = str( self.special_abilities_name[i] )
            specialAbilityDesc = str( self.special_abilities_desc[i] )
            specialAbilityBonus = int( self.special_abilities_attack_bonus[i] )

            print('The '+name+' uses its '+specialAbilityName)
            print( specialAbilityDesc )
            print('')

            
            

In [751]:
monst1 = Monster(is_random=True)
monst1_index = monst1.index
monst1.perform_random_action()
monst1.perform_random_special_ability()

The Vampire uses its Charm
The vampire targets one humanoid it can see within 30 ft. of it. If the target can see the vampire, the target must succeed on a DC 17 Wisdom saving throw against this magic or be charmed by the vampire. The charmed target regards the vampire as a trusted friend to be heeded and protected. Although the target isn't under the vampire's control, it takes the vampire's requests or actions in the most favorable way it can, and it is a willing target for the vampire's bit attack.
Each time the vampire or the vampire's companions do anything harmful to the target, it can repeat the saving throw, ending the effect on itself on a success. Otherwise, the effect lasts 24 hours or until the vampire is destroyed, is on a different plane of existence than the target, or takes a bonus action to end the effect.

The Vampire uses its Regeneration
The vampire regains 20 hit points at the start of its turn if it has at least 1 hit point and isn't in sunlight or running water. 